In [14]:
#Libraries
import os
import numpy as np
import pandas as pd
import cv2
from sklearn.model_selection import train_test_split
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras.regularizers import l2
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.python.client import device_lib

#visualizations to be used.... maybe.
import seaborn as sns
import matplotlib.pyplot as plt

In [15]:

def train_id_to_path(x):
    return '../petfinder-pawpularity-score/train/' + x + ".jpg"
def test_id_to_path(x):
    return '../petfinder-pawpularity-score/test/' + x + ".jpg"


train = pd.read_csv('../petfinder-pawpularity-score/train2.csv')
train = train.drop(['Subject Focus', 'Eyes', 'Face', 'Near', 'Action', 'Accessory', 'Group', 'Collage', 'Human', 'Occlusion', 'Info', 'Blur'],axis=1)

test = pd.read_csv('../petfinder-pawpularity-score/test2.csv')
test = test.drop(['Subject Focus', 'Eyes', 'Face', 'Near', 'Action', 'Accessory', 'Group', 'Collage', 'Human', 'Occlusion', 'Info', 'Blur'],axis=1)

train["img_path"] = train["Id"].apply(train_id_to_path)
test["img_path"] = test["Id"].apply(test_id_to_path)

In [16]:
#EDA
#Set the size image to 128 x 128
image_height = 128
image_width = 128

def path_to_eagertensor(image_path):
    '''
    Function that accepts an img url and returns an eager tensor
    '''
    raw = tf.io.read_file(image_path)
    image = tf.image.decode_jpeg(raw, channels=3)
    image = tf.cast(image, tf.float32) / 255.0
    image = tf.image.resize(image, (image_height, image_width))
    return image

X = []
for img in train['img_path']: #Converting the paths to eager tensors into array X
    new_img_tensor = path_to_eagertensor(img)
    X.append(new_img_tensor)
    

X = np.array(X)

## MODEL TIME

In [17]:
y = train['Pawpularity']
#print(type(y))

x_train, x_test, y_train, y_test = train_test_split(X, y, test_size=0.1, random_state=12345) #Splitting the data into the appropriate bins for x and y training and test
print("X train shape:",x_train.shape, "\nX test shape:", x_test.shape, "\nY train shape:", y_train.shape, "\nY test shape:", y_test.shape)

X train shape: (8913, 128, 128, 3) 
X test shape: (991, 128, 128, 3) 
Y train shape: (8913,) 
Y test shape: (991,)


In [19]:
#define the inputs (shape of the incoming data)
inputs = tf.keras.Input(shape=(image_height,image_width,3))

x = inputs
x = tf.keras.layers.Conv2D(filters = 16, kernel_size = (7,7), strides = (2,2), padding='valid', kernel_initializer='he_normal', kernel_regularizer=l2(0.0005), activation = 'relu')(x)
x = tf.keras.layers.Conv2D(filters = 32, kernel_size = (3,3), padding='same', activation = 'relu')(x)
x = tf.keras.layers.BatchNormalization()(x)
x = tf.keras.layers.Conv2D(filters = 32, kernel_size = (3,3), strides = (2,2), padding='same', kernel_initializer='he_normal', kernel_regularizer=l2(0.0005), activation = 'relu')(x)
x = tf.keras.layers.BatchNormalization()(x)
x = tf.keras.layers.Dropout(0.25)(x)


x = tf.keras.layers.Conv2D(filters = 64, kernel_size = (3,3), padding='same', kernel_initializer='he_normal', kernel_regularizer=l2(0.0002), activation = 'relu')(x)
x = tf.keras.layers.BatchNormalization()(x)
x = tf.keras.layers.Conv2D(filters = 64, kernel_size = (3,3), strides = (2,2), padding='same', kernel_initializer='he_normal', kernel_regularizer=l2(0.0005), activation = 'relu')(x)
x = tf.keras.layers.BatchNormalization()(x)
x = tf.keras.layers.Dropout(0.25)(x)

x = tf.keras.layers.Conv2D(filters = 128, kernel_size = (3,3), padding='same', kernel_initializer='he_normal', kernel_regularizer=l2(0.0002), activation = 'relu')(x)
x = tf.keras.layers.BatchNormalization()(x)
x = tf.keras.layers.MaxPooling2D(pool_size=(2, 2))(x)
x = tf.keras.layers.Conv2D(filters = 128, kernel_size = (3,3),padding='same', kernel_initializer='he_normal', kernel_regularizer=l2(0.0002), activation = 'relu')(x)
x = tf.keras.layers.BatchNormalization()(x)
x = tf.keras.layers.Dropout(0.25)(x)

x = tf.keras.layers.Flatten()(x)
x = tf.keras.layers.Dense(512, activation = "relu")(x)
x = tf.keras.layers.Dropout(0.5)(x)

output = tf.keras.layers.Dense(1)(x)

model = tf.keras.Model(inputs = inputs, outputs = output)

#MODEL COMPILATION TIME
model.compile(
    loss = 'mse', 
    optimizer = 'Adam', 
    #metrics = [tf.keras.metrics.RootMeanSquaredError(name="rmse"), "mae", "mape"])   <-- Don't understand this output
    metrics =  ['accuracy', 'mse']) # <-- So I picked this one

data_augmentation = ImageDataGenerator( #Slightly changes each image by randomizing certain variables creating slight variations per epoch
    rotation_range = 15, 
    zoom_range = 0.15,
    width_shift_range = 0.2, 
    height_shift_range = 0.2, 
    shear_range = 0.1,
    horizontal_flip = True, 
    fill_mode = "nearest")

history = model.fit( #Final model fit
    data_augmentation.flow(x_train,y_train,batch_size=32),
    validation_data = (x_test,y_test),
    steps_per_epoch = len(x_train) // 32,
    epochs = 3
)

score = model.evaluate(x_test, y_test, verbose=1)

print(f"Model Score: {score}")

Epoch 1/3
278/278 [==============================] - 58s 207ms/step - loss: 503.2297 - accuracy: 4.5040e-04 - mse: 502.9815 - val_loss: 428.2518 - val_accuracy: 0.0000e+00 - val_mse: 427.9980
Epoch 2/3
278/278 [==============================] - 56s 203ms/step - loss: 476.3538 - accuracy: 4.5040e-04 - mse: 476.0965 - val_loss: 406.4117 - val_accuracy: 0.0000e+00 - val_mse: 406.1512
Epoch 3/3
31/31 [==============================] - 1s 35ms/step - loss: 422.1701 - accuracy: 0.0000e+00 - mse: 421.9049
Model Score: [422.17010498046875, 0.0, 421.9049072265625]


In [20]:
print(f"Model Score: {score[1]}")

Model Score: 0.0
